## Word Embeddings
### Prabhjot Singh
### DS 5001
### 10 May 2021

In [2]:
# packages used
import pandas as pd
import numpy as np
from gensim.models import word2vec
from sklearn.manifold import TSNE
import plotly_express as px


In [42]:
# defining the OHCO
# setting OHCO
OHCO = ["EVENT_Label","EVENT_ID", "para_num", "sent_num", "token_num"] # event ID is the chapter number 

# setting the bags
SENTS = OHCO[:4]
PARAS = OHCO[:3]
EVENT_ID = OHCO[:2]
EVENT_TYPE = OHCO[:1]

BAG = SENTS

In [43]:
LIB = pd.read_csv("./data_files/LIB2020.csv")
LIB_Types = pd.read_csv("./data_files/LIB_TYPES2020.csv", index_col=[0])
TOKEN = pd.read_csv("./data_files/TOKEN2020.csv").set_index(OHCO)
VOCAB = pd.read_csv("./data_files/2020vocab_tfidf.csv").set_index("term_str")

In [44]:
VOCAB

,term_id,n,num,stop,p_stem,pos_max,df,idf,tfidf_sum_event_max
term_str,,,,,,,,,
0,1,17,1,0,0,CD,17,11.412819,68.476913
000,2,1,1,0,000,CD,1,15.500282,1.409117
0000,3,11,1,0,0000,CD,11,12.040850,88.012880
0000cst,4,4,1,0,0000cst,CD,4,13.500282,15.187817
0001,5,6,1,0,0001,CD,6,12.915319,30.135745
...,...,...,...,...,...,...,...,...,...
zoologico,28971,1,0,0,zoologico,NNP,1,15.500282,15.500282
zortman,28972,11,0,0,zortman,NNP,11,12.040850,81.198553
zucksville,28973,1,0,0,zucksvil,NNP,1,15.500282,15.500282


In [45]:
TOKEN

pos_tuple  pos  \
EVENT_Label EVENT_ID para_num sent_num token_num                               
1           863438   0        0        0          ('Visibility', 'NNP')  NNP   
                                       1                ('below', 'IN')   IN   
                                       2                    ('a', 'DT')   DT   
                                       3              ('quarter', 'NN')   NN   
                                       4                 ('mile', 'NN')   NN   
...                                                                 ...  ...   
11          932481   0        1        6           ('Spaulding', 'NNP')  NNP   
                                       7                 ('also', 'RB')   RB   
                                       8                   ('in', 'IN')   IN   
                                       9              ('Lassen', 'NNP')  NNP   
                                       10            ('County.', 'NNP')  NNP   

                                                   token_str    term_str  
EVENT_Label EVENT_ID para_num sent_num token_num                          
1           863438   0        0        0          Visibility  visibility  
                                       1               below       below  
                                       2                   a           a  
                                       3             quarter     quarter  
                                       4                mile        mile  
...                                                      ...         ...  
11          932481   0        1        6           Spaulding   spaulding  
                                       7                also        also  
                                       8                  in          in  
                                       9              Lassen      lassen  
                                       10            County.      county  

[1149191 rows x 4 columns]

In [46]:
# # making doc table
DOCS = TOKEN[~TOKEN.pos.str.match('NNPS?')]\
    .groupby(BAG)\
    .term_str.apply(lambda  x:  x.tolist())\
    .reset_index()['term_str'].tolist()
DOCS = [doc for doc in DOCS if len(doc) > 1] # Lose single word docs

In [47]:
DOCS = str(DOCS)

In [48]:
# DOCS

In [49]:
# making the model
w2v_params_2020 = dict(
        window = 5,
        size = 246,
        min_count = 100,
        workers = 4)

model_2020 = word2vec.Word2Vec(DOCS, **w2v_params_2020)


In [28]:
# model_2020

In [37]:
def t_sne(model, VOCAB):
    
    coords = pd.DataFrame(
    dict(
        vector = [model.wv.get_vector(w) for w in model.wv.vocab], 
        term_str = model.wv.vocab.keys()
    )).set_index('term_str')
    
    
    tsne_engine = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    tsne_model = tsne_engine.fit_transform(coords.vector.to_list())
    
    
    coords['x'] = tsne_model[:,0]
    coords['y'] = tsne_model[:,1]
    
    if coords.shape[1] == 3:
        coords = coords.merge(VOCAB.reset_index(), on='term_str')
        coords = coords.set_index('term_str')
        
    coords = coords[coords.stop == 0]
    
    fig = px.scatter(coords.reset_index(), 'x', 'y', 
           text='term_str', 
           color='pos_max', 
           hover_name='term_str',          
           size='tfidf_sum_event_max',
           height=1000).update_traces(
                mode='markers+text', 
                textfont=dict(color='black', size=14, family='Arial'),
                textposition='top center')
    fig.show()

In [39]:
VOCAB

,term_id,n,num,stop,p_stem,pos_max,df,idf,tfidf_sum_event_max
term_str,,,,,,,,,
0,1,17,1,0,0,CD,17,11.412819,68.476913
000,2,1,1,0,000,CD,1,15.500282,1.409117
0000,3,11,1,0,0000,CD,11,12.040850,88.012880
0000cst,4,4,1,0,0000cst,CD,4,13.500282,15.187817
0001,5,6,1,0,0001,CD,6,12.915319,30.135745
...,...,...,...,...,...,...,...,...,...
zoologico,28971,1,0,0,zoologico,NNP,1,15.500282,15.500282
zortman,28972,11,0,0,zortman,NNP,11,12.040850,81.198553
zucksville,28973,1,0,0,zucksvil,NNP,1,15.500282,15.500282


In [38]:
t_sne(model_2020, VOCAB)